In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts( ["Cats love thuna"], embedding=OpenAIEmbeddings() )
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context: {context} Question: {question} """
prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = ( {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | ChatOpenAI() | StrOutputParser() )

It is not strictly mandatory for every single element in a LangChain Expression Language (LCEL) chain to implement the Runnable interface, but the core structure relies heavily on this protocol for consistency and functionality.

Fundamental LangChain components like LLMs (`ChatOpenAI`), prompt templates (`ChatPromptTemplate`), retrievers, vector stores, and output parsers all implement the Runnable interface by default.

Custom Functions are Adaptable: When you use a standard Python function in a chain, like the format_docs function in example code above, LangChain automatically adapts it into a Runnable internally. It wraps the function with a utility that implements the necessary interface, allowing it to seamlessly integrate into the chain.

Structure Elements: Certain structural elements, such as dictionaries (`{"context": ..., "question": ...}` in your example), are also implicitly handled by the framework to route inputs correctly within the chain structure, adhering to the overall protocol.

You do not explicitly need to wrap a simple, synchronous custom Python function with `RunnableLambda` when using the LangChain Expression Language (LCEL) chains. LangChain automatically detects standard Python functions used within the | syntax (the pipe operator) and wraps them internally to conform to the Runnable protocol.

`RunnableLambda` is primarily useful in specific, more advanced scenarios:

**Explicit Naming/Configuring**: To explicitly name a step in the chain for better tracing and debugging in tools like LangSmith.

**Asynchronous Functions**: When integrating custom async functions (those defined with async def), you might use `RunnableLambda(..., affordances=[...])` to ensure asynchronous capabilities are correctly exposed.

**Method Calls**: To wrap methods from a class instance that need to be part of the chain.
For everyday use with basic synchronous functions, you can rely on the automatic conversion provided by the framework.

### 1. Explicit Naming for Tracing (LangSmith)
When you use a standard Python function in a chain, it often appears as a generic "function" or the function name itself in tracing tools like LangSmith. RunnableLambda allows you to assign a descriptive, user-friendly name to that specific step in the trace.

In [ ]:
from langchain_core.runnables import RunnableLambda
# Assume 'docs' is a list of Document objects

def format_docs_func(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Wrap the function and assign a specific name using .with_config()
format_docs_step = RunnableLambda(format_docs_func).with_config(
    run_name="FormatDocumentsStep"
)

# In the trace visualization:
# The step that runs format_docs_func will now be labeled "FormatDocumentsStep"
# instead of just "format_docs_func" or "function".
chain = (
    # ... retriever step ...
    | format_docs_step
    # ... prompt and model steps ...
)


# 2. Handling Asynchronous Functions
LangChain chains support asynchronous execution (.ainvoke() or .abatch()). While synchronous functions work automatically, async def functions require RunnableLambda to correctly expose their asynchronous nature to the chain framework.

In [ ]:
import time
from langchain_core.runnables import RunnableLambda
import asyncio

# A standard synchronous function
def sync_processing(text):
    time.sleep(1) # Simulate time-consuming I/O
    return f"Processed sync: {text}"

# An asynchronous function
async def async_processing(text):
    await asyncio.sleep(1) # Use await for async operations
    return f"Processed async: {text}"

# The sync function works inline automatically:
sync_runnable = sync_processing

# The async function *must* be wrapped explicitly with RunnableLambda to be callable asynchronously
async_runnable = RunnableLambda(async_processing)

# The chain can now correctly use .ainvoke() with the async component:
async def run_async_chain():
    result = await async_runnable.ainvoke("data_input")
    print(result)

# When you execute run_async_chain(), the chain knows how to call the async function correctly.


# 3. Wrapping Method Calls from a Class Instance
If you are using object-oriented programming and want to integrate a specific method of a class instance into a chain, RunnableLambda is the cleanest way to reference that method specifically.

In [ ]:
from langchain_core.runnables import RunnableLambda

class DataProcessor:
    def __init__(self, prefix):
        self.prefix = prefix

    def process_data(self, data: str) -> str:
        """A method that uses instance state (self.prefix)."""
        return f"{self.prefix}: {data.upper()}"

processor_instance = DataProcessor(prefix="INFO")

# Wrap the specific instance method `processor_instance.process_data`
process_step = RunnableLambda(processor_instance.process_data)

# This method is now a Runnable and can be used in a chain:
final_chain = (
    RunnableLambda(lambda x: x + " raw_input")
    | process_step
)

# Example usage:
output = final_chain.invoke("My")
print(output)
# Output: INFO: MY RAW_INPUT
